# McStas to NeXus for ESTIA

This notebook converts data from a McStas simulation output (`.h5` filetype) to a NeXus file that uses a file for the Estia instrument (written by CODA) as a template for the geometry information.

It adds

* events to the `multiblade_detector` detector,
* a single value to the `proton_current` log,
* a single detector rotation value to the `detector_rotation` log,
* a single sample rotation value to the `sample_rotation` log.

In [ ]:
import numpy as np
import scipp as sc
import h5py
import shutil

from ess.estia.mcstas import parse_events_h5
from ess.estia.load import load_mcstas
from ess.estia.data import estia_mcstas_example



In [ ]:
def replace_dataset(entry, name, values):
    attrs = dict(entry[name].attrs)
    del entry[name]
    dset = entry.create_dataset(name, data=values)
    dset.attrs.update(attrs)


def mcstas_to_nexus(
    mcstas_data_file: str,
    template_nexus_file: str,
    outfile: str,
    nevents: int = 1_000_000,
):
    """
    Store the events from a McStas Estia simulation in a NeXus CODA file.

    Parameters
    ----------
    mcstas_data_file:
        Data file containing simulated McStas events.
    template_nexus_file:
        NeXus file containing geometry and instrument info, used as a template.
    outfile:
        Output file to be written.
    nevents:
        Number of events to have in the output file
        (events are sampled from the probabilities of the mcstas events).
    """
    detector_entry_path = '/entry/instrument/multiblade_detector'

    with h5py.File(template_nexus_file, "r") as f:
        det_numbers = f[f"{detector_entry_path}/detector_number"][()]

    with h5py.File(mcstas_data_file) as f:
        events = parse_events_h5(f, nevents_to_sample=nevents)
    binned = load_mcstas(events).transpose(['strip', 'blade', 'wire', ])

    toa = (binned.bins.coords["event_time_zero"] + binned.bins.coords["event_time_offset"]).bins.concat().value

    det_numbers_matching_order_of_binned = det_numbers.reshape(binned.shape)
    det_numbers_matching_order_of_binned = det_numbers_matching_order_of_binned[::-1, ::-1, :]
    # IMPORTANT! we need to sort the arrays below according to toa,
    # so that the event_index does not get messed up!
    event_id = sc.sort(
        (
            sc.bins_like(binned, sc.array(dims=binned.dims, values=det_numbers_matching_order_of_binned))
            .bins.concat()
            .value
        ),
        key=toa,
    )
    event_time_zero = sc.sort(binned.bins.coords["event_time_zero"].bins.concat().value, key=toa)
    event_time_offset = sc.sort(binned.bins.coords["event_time_offset"].bins.concat().value, key=toa)

    event_index = sc.DataArray(
        data=sc.ones_like(event_time_offset),
        coords={"event_time_zero": event_time_zero},
    ).group("event_time_zero")

    event_index = sc.cumsum(event_index.bins.size())
    event_index.values = np.concatenate([[0], event_index.values[:-1]])

    shutil.copyfile(template_nexus_file, outfile)

    with h5py.File(outfile, "r+") as f:

        detector_rotation = f[f"{detector_entry_path}/transformations/detector_rotation"]
        replace_dataset(
            detector_rotation,
            name="value",
            values=[
                binned
                    .coords['detector_rotation']
                    .to(unit=detector_rotation['value'].attrs['units'])
                    .value
            ]
        )
        replace_dataset(
            detector_rotation,
            name="time",
            values=[
                event_time_zero
                    .min()
                    .to(unit=detector_rotation['time'].attrs['units'])
                    .value
                    .astype('uint64')
            ]
        )

        # TODO: Put sample_rotation below "transformations" group
        # TODO: Set correct nexus classes on the sample_rotation group
        sample = f["entry/sample"]
        sample_rotation = sample.create_group('sample_rotation')
        sample_rotation.create_dataset(
            'value',
            data=[
                binned
                .coords['sample_rotation']
                .to(unit='deg')
                .value
                .astype('float64')
            ],
        )
        sample_rotation['value'].attrs['units'] = 'degrees'
        sample_rotation.create_dataset(
            'time',
            data=[
                event_time_zero
                    .min()
                    .to(unit='ns')
                    .value
                    .astype('uint64')
            ],
        )
        sample_rotation['time'].attrs['units'] = 'ns'

        f["entry/neutron_prod_info/pulse_charge/value"][:] = 1.

        event_data = f[f"{detector_entry_path}/event_data"]
        replace_dataset(event_data, name="event_id", values=event_id.values.astype('uint32'))
        replace_dataset(
            event_data,
            name="event_time_offset",
            values=event_time_offset.to(
                unit=event_data["event_time_offset"].attrs["units"], copy=False
            ).values.astype('uint32'),
        )
        replace_dataset(event_data, name="event_index", values=event_index.values.astype('uint64'))
        replace_dataset(
            event_data,
            name="event_time_zero",
            values=event_index.coords["event_time_zero"]
            .to(unit=event_data["event_time_zero"].attrs["units"], copy=False)
            .values.astype('uint64'),
        )

In [ ]:

mcstas_to_nexus(
    mcstas_data_file=estia_mcstas_example('reference'),
    template_nexus_file="/home/johannes/Downloads/estia_999999_00008786.hdf",
    outfile='reference.nx',
    nevents=10_000_000,
)

mcstas_to_nexus(
    mcstas_data_file=estia_mcstas_example('Ni/Ti-multilayer')[1],
    template_nexus_file="/home/johannes/Downloads/estia_999999_00008786.hdf",
    outfile='niti_ml_1.nx',
    nevents=10_000_000,
)

In [ ]:
import scippneutron as scn
from ess.reflectometry.types import *
from ess.estia import EstiaWorkflow, EstiaMcStasWorkflow

wfm = EstiaMcStasWorkflow()
wfm[Filename[SampleRun]] = estia_mcstas_example('Ni/Ti-multilayer')[1]

scn.instrument_view(wfm.compute(RawDetector[SampleRun]).hist(), pixel_size=0.003, norm='log')

In [ ]:
import scippneutron as scn
wf = EstiaWorkflow()
wf[Filename[SampleRun]] = 'niti_ml_1.nx'

scn.instrument_view(wf.compute(RawDetector[SampleRun]).hist(), pixel_size=3, norm='log')

In [ ]:
from ess.reflectometry.types import *
from ess.estia import EstiaWorkflow, EstiaMcStasWorkflow

n = 20_000_000

wfm = EstiaMcStasWorkflow()
wfm[Filename[SampleRun]] = estia_mcstas_example('reference')
h = wfm.compute(RawDetector[SampleRun]).flatten(['blade', 'wire'], 'x').bins.sum()
p1 = sc.round(sc.values(0.9 * n * h / h.sum().value)).plot(norm='log')

wf = EstiaWorkflow()
wf[Filename[SampleRun]] = 'out.nx'
p2 = wf.compute(RawDetector[SampleRun]).flatten(['blade', 'wire'], 'x').transpose().bins.sum().plot(norm='log')

p1 + p2

In [ ]:
wfm.compute(RawDetector[SampleRun]).bins.concat().hist(event_time_offset=200).plot()  + wf.compute(RawDetector[SampleRun]).bins.concat().hist(event_time_offset=200).plot()

In [ ]:
%matplotlib ipympl
sc.round(sc.values(0.9* n * h / h.sum().value)).plot(norm='log')

In [ ]:
wf.compute(RawDetector[SampleRun]).flatten(['blade', 'wire'], 'x').transpose().bins.sum().plot(norm='log')

In [ ]:
import scipp as sc

from ess.estia.data import estia_mcstas_example, estia_tof_lookup_table
from ess.estia import EstiaWorkflow
from ess.reflectometry.types import *
from ess.reflectometry import supermirror

wf = EstiaWorkflow()
wf[Filename[SampleRun]] = 'niti_ml_1.nx'
wf[Filename[ReferenceRun]] = 'reference.nx'
# Select a region of interest:
wf[YIndexLimits] = sc.scalar(35), sc.scalar(64)
wf[ZIndexLimits] = sc.scalar(0), sc.scalar(48 * 32)
wf[BeamDivergenceLimits] = sc.scalar(-0.75, unit='deg'), sc.scalar(0.75, unit='deg')

wf[supermirror.MValue] = sc.scalar(5, unit=sc.units.dimensionless)
wf[supermirror.CriticalEdge] = sc.scalar(float('inf'), unit='1/angstrom')
wf[supermirror.Alpha] = sc.scalar(0.25 / 0.088, unit=sc.units.angstrom)

wf[DetectorSpatialResolution[RunType]] = 0.0025 * sc.units.m

# Configure the binning of intermediate and final results:
wf[WavelengthBins] = sc.geomspace('wavelength', 3.5, 12, 2001, unit='angstrom')
wf[QBins] = 1000

wf[TimeOfFlightLookupTableFilename] = estia_tof_lookup_table()

wf[ProtonCurrent[SampleRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})
wf[ProtonCurrent[ReferenceRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})

wf.visualize(ReflectivityOverQ)

In [ ]:
da = wf.compute(ReducibleData[SampleRun])
da

In [ ]:
import scippneutron as scn

scn.instrument_view(da.hist(), pixel_size=3, norm='log')

In [ ]:
from ess.reduce.nexus.types import DetectorBankSizes
wf.compute(DetectorBankSizes)